# Practical 2 : Generative and Discriminative Models


In this practical, we will compare the Naïve Bayes Classifier (NBC) and Logistic Regression on several
datasets. As part of the practical you should read briefly the following paper:



**On Discriminative vs. Generative classifiers: A comparison of logistic regression
and naive Bayes**  
*Andrew Y. Ng and Michael I. Jordan*  
Advances in Neural Information Processing Systems (NIPS) 2001.

The paper is available on OLAT. 

You should read the Introduction and the Experiments sections. The goal of this practical is
to qualitatively reproduce some of the experimental results in this paper. You are strongly
encouraged to read the rest of the paper, which is rather short and straightforward to read,
though some of you may want to skip the formal proofs.

## Naïve Bayes Classifier (NBC)

You should implement a Naïve Bayes Classifier from scartch using NumPy. To keep your code tidy,
we recommend implementing it as a class. 
The classifier should be able to handle binary and continuous features. 
To earn the bonus points, the classifier should be able to handle categorical features as well. 
Suppose the data has 3
different features, the first being binary, the second being continuous and the third being categorical. Write an implementation that you can initialise as follows:

    nbc = NBC(feature_types=['b', 'r', 'c'])

Along the lines of classifiers provided in sklearn, you want to implement two more functions,
**fit** and **predict**. 
Recall the joint distribution of a generative model: $p(\mathbf{x}, y \mid \theta, \pi) = p(y \mid \pi) \cdot p(\mathbf{x} \mid y, \theta)$.
The **fit** function is to estimate all the parameters ($\theta$ and $\pi$) of the NBC, i.e., train the classifier. The **predict** function computes the probabilities that the new input belongs to all classes and
then returns the class that has the largest probability, i.e., makes a prediction.

    nbc.fit(X_train, y_train)
    ypredicted = nbc.predict(X_test)
    test_accuracy = np.mean(ypredicted == ytest)

Here we import the libraries. 

In [123]:
import pandas as pd
import scipy.stats
from scipy.stats import bernoulli
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (10., 10.)

import pickle as cp
import matplotlib.pyplot as plt
import numpy as np


### Class-conditional distributions

Before implementing NBC, we first implement the class-conditional distributions $p(\mathbf{x} \mid y, \theta)$. Your implementation should have two functions: **estimate** and **get_log_probability**. 

- The **estimate** function takes data as input and models the data using some distribution $p(x \mid \theta)$, where $\theta$ is the parameters of this distribution. The function estimates the parameters $\theta$ using maximum likelihood estimators (MLE). 
For example, in the case of continuous features, we use the Gaussian distribution to model the data. The estimate function will find the parameters $\mu$ and $\sigma$ for the Gaussian distribution with respect to the input data. 

- The **get_log_probability** function takes as input a new data point $x_{new}$ and returns the log of $p(x_{new} \mid \theta)$. For the Gaussian distribution, the function get_probability will return $\mathcal{N}(x_{new} \mid \mu, \sigma^2)$. 

For different types of features, you need to use different distributions.
You can import statistic libraries (e.g., `scipy.stats`) for the implementation of the distributions. 

- For **continuous features**: Use Gaussian distribution
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html
- For **binary features**: Use Bernoulli distribution 
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bernoulli.html
- For **categorical features**: Use Multinoulli distribution (The multinoulli distribution is a special case of the multinomial distribution, where the number of trials is 1)
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multinomial.html

Example: https://stackoverflow.com/questions/12412895/how-to-calculate-probability-in-a-normal-distribution-given-mean-standard-devi


**Implementation Issues:**
- The probabilities can be very small. To avoid underflow issues, you should compute the log of the probabilities. Read more: (Mur) Chapter 3.5.3 / Lecture Notes
- The variance for Gaussian distributions should never be exactly 0, so in
case your calculated variance is 0, you may want to set it to a small value such as 1e − 6. This is to ensure that your code never encounters division by zero or
taking logarithms of 0 errors. 
For this practical, please set the small value to 1e-6.
- Laplace/Additive smoothing: You want to ensure that the estimates for the parameter for the Bernoulli and Multinoulli random variables is never exactly 0 or 1. For this reason you should consider using Laplace smoothing (https://en.wikipedia.org/wiki/Additive_smoothing).
For this practical, please set alpha to 1.
- For simplicity, you can assume the data values for binary features are integers from {0,1} and the data for a categorical feature with M categories are integers from {0, ..., M-1}.
- Fell free to add auxiliary functions or change the parameters of the functions. If you change the parameters of the functions, make sure you change the tests accordingly, so we can test your code.


In [217]:
ALPHA = 1.0 # for additive smoothing
SMALL_VALUE = 1e-6

# Distribution for continuous features
class ContFeatureParam:
    def __init__(self) -> None:
        self.normal_dist = None
    def estimate(self, X):
        # TODO: Estimate the parameters for the Gaussian distribution 
        # so that it best describes the input data X
        X_mean = np.mean(X)
        X_std_div = np.std(X)
        # check if variance is 0 and set it to small value 
        if(X_std_div == 0):
            X_std_div = SMALL_VALUE
        #my_normal_dist = 1/(np.sqrt(2*np.pi) * X_std_div) * \
        #    np.exp(-0.5*(((X-X_mean)/X_std_div)**2))
        normal_dist = scipy.stats.norm(X_mean, X_std_div)
        self.normal_dist = normal_dist
        
        return self

    def get_log_probability(self, X_new):
        # TODO: return the log of the density values for the input values X_new
        # The code below is just for compilation. 
        # You need to replace it by your own code.
        if self.normal_dist:
            return self.normal_dist.pdf(X_new)
        else: 
            raise Exception("Please run the estimate function first")

# Distribution for binary features
class BinFeatureParam:
    def __init__(self) -> None:
        self.p1 = None
    def estimate(self, X):
        # TODO: Estimate the parameters for the Bernoulli distribution 
        # so that it best describes the input data X
        # X is a vector with binary values X = np.array([0,0,1,1,0,1,0,1,1,1])
        count_ones = np.count_nonzero(X==1)
        array_size = X.size
        # without Laplace smoothing
        #p1 = count_ones/array_size
        
        #Laplace smoothing
        p1 = (count_ones + ALPHA)/(array_size + 2*ALPHA)
        
        #print(p1)
        
        self.p1 = p1
        
        return self

    def get_log_probability(self, X_new):
        # TODO: return the log of the probability values for the input values X_new
        
        # convert X_new to float to accept the probability
        #print(bernoulli.logpmf(X_new,self.p1))
        X_new = X_new.astype(np.float32)
        if(self.p1):
            #X_new[X_new == 1] = self.p1
            #X_new[X_new == 0] = 1 - self.p1
            #return np.log(X_new)
            return bernoulli.pmf(X_new, self.p1)
        else:
            raise Exception("Please run the estimate function first")



# Distribution for categorical features
class CatFeatureParam:
    
    # we need to know the number of categories for the categorical feature
    def __init__(self, num_of_categories):
        self._num_of_categories = num_of_categories
        self.prob = None
    
    def estimate(self, X):
        # TODO: Estimate the parameters for the Multinoulli distribution
        # so that it best describes the input data X
        
        prob = {}
        # initialize our dict 
        for i in range(self._num_of_categories):
            prob.update({i: 0})
            
        unique, counts = np.unique(X, return_counts=True)
        # without laplace 
        # counts = counts / X.size
        #assert counts.sum() == 1

        prob.update(dict(zip(unique, counts)))
        # Laplace smoothing
        for key, val in prob.items():
            prob[key] = (val + ALPHA)/(self._num_of_categories*ALPHA + X.size)
        
        assert sum(prob.values()) == 1
       

            
        #print(prob)
        self.prob = prob
        return self


    def get_log_probability(self, X_new):
        # TODO: return the log of the probability values for the input values X_new
        if self.prob:
            return np.array([self.prob.get(key) for key in X_new])
            


**Tests:**
    
We will use the code below to test the correctness of your code.

In [196]:
# continuous features

X = np.array([2.70508547,2.10499698,1.76019132,3.42016431,3.47037973,3.67435061,1.84749286,4.3388506,2.27818252,4.65165335])

param = ContFeatureParam()
param.estimate(X)
probs = param.get_log_probability(np.array([0,1,2,3]))
print(probs)

[0.00352912 0.04843274 0.23558711 0.40616623]


In [203]:
# binary features

X = np.array([0,0,1,1,0,1,0,1,1,1])



param = BinFeatureParam()
param.estimate(X)
probs = param.get_log_probability(np.array([0,1]))
print(probs)

[0.41666667 0.58333333]


In [221]:
# categorical features (bonus task)

X = np.array([0,6,5,4,0,6,6,4,1,1,2,3,8,8,1,6,4,9,0,2,2,3,8,0,2])
param = CatFeatureParam(max(X)+1)
param.estimate(X)
probs = param.get_log_probability(np.array([0,1,2,3,4,5,6,7,8,9]))
print(probs)

[0.14285714 0.11428571 0.14285714 0.08571429 0.11428571 0.05714286
 0.14285714 0.02857143 0.11428571 0.05714286]


### Implement NBC

We are now ready to implement NBC. We follow the structure of models in scikit-learn. We implement NBC as a class with functions **init**, **fit** and **predict**.
The **init** function takes as input the types of features and initialise the classifier. The **fit** function takes the training data as input and estimates the parameters. The **predict** function predicts the label for the input data. 

**Implementation Issues:**
- You should use matrix operations rather than loops. In general, loops over classes or features are OK, but loops over the rows of data are not a good idea.
- The probabilities can be very small. To avoid underflow issues, you should do the calculations in log space. Read more: (Mur) Chapter 3.5.3 / Lecture Note
- For simplicity, you can assume the data values for binary features are integers from {0, 1} and the data for a categorical feature with M categories are integers from {0, ..., M-1}.
- Fell free to add auxiliary functions or change the parameters of the functions. If you change the parameters of the functions, make sure you change the tests accordingly, so we can test your code.

In [215]:
# Your task is to implement the three functions of NBC. 

class NBC:
    # Inputs:
    #   feature_types: the array of the types of the features, e.g., feature_types=['b', 'r', 'c']
    def __init__(self, feature_types=[]):
        self.feature_types = feature_types
        # make sure at least 1 feature type is passed
        assert len(self.feature_types) >=1
        self.class_prior = None
        self.estimation_all_classes = {}
        self.unique_classes = []
     

        
        
        


        
    # The function uses the input data to estimate all the parameters of the NBC
    def fit(self, X, y):
        # compute class prior
        # get count of unique classes
        unique_classes, counts = np.unique(y, return_counts=True)
        self.unique_classes = unique_classes
        # compute appearance of each class in the data
        counts = counts / y.size
        self.class_prior = dict(zip(unique_classes,counts))
        print(self.class_prior)
        
        # Estimate a parameter 𝜃𝑗𝑐 for each class 𝑐 and each feature 𝑗
        for c in self.unique_classes:
            estimations_c = []
            for j in range(len(self.feature_types)):
                # get each column (feature) of class c 
                column_j_c = X[y == c][:, j]
                
                # estimate each column and add to array
                
                # choose prober estimator
                if (self.feature_types[j] == 'r'):
                    estimations_c.append(ContFeatureParam().estimate(column_j_c))
                elif (self.feature_types[j] == 'b'):
                    estimations_c.append(
                        BinFeatureParam().estimate(column_j_c))
                elif (self.feature_types[j] == 'c'):
                    estimations_c.append(
                        CatFeatureParam(max(column_j_c)).estimate(column_j_c))

            # add estimation of class c to our main estimation array
            self.estimation_all_classes.update({c:estimations_c})
        
        #print(self.estimation_all_classes)
                
        


        
                
                
    # The function takes the data X as input, and predicts the class for the data
    def predict(self, X):
        # for predict we just take the 𝑝(𝑦 = 𝑐|𝝅)=𝜋c
        df = pd.DataFrame()
        loc_for_df = 0
        for c in self.unique_classes:
            pi_c = self.class_prior[c]
            prob_c = np.ones(shape=(X.shape))
            #print(prob_c.shape)
            for j in range(len(self.feature_types)):
                column_j = X[:, j]
                prob_c[:,j] = self.estimation_all_classes[c][j].get_log_probability(column_j)
            df.insert(loc_for_df, c,(np.prod(prob_c, axis=1)*pi_c))
            loc_for_df+=1
        df["prediction"] = df.idxmax(axis=1)
        #display(df)
        return df["prediction"]


            


**Tests**

We will use the code below to check your code.

In [208]:
# All features of the iris dataset are continuous.
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris['data'], iris['target']

df = pd.DataFrame(data=X)
df.insert(4,4, y)
# displaying the dataframe
#display(df)

N, D = X.shape
Ntrain = int(0.8 * N)
Xtrain = X[:Ntrain]
ytrain = y[:Ntrain]
Xtest = X[Ntrain:]
ytest = y[Ntrain:]


nbc_iris = NBC(feature_types=['r', 'r', 'r', 'r'])
nbc_iris.fit(Xtrain, ytrain)
yhat = nbc_iris.predict(Xtest)
test_accuracy = np.mean(yhat == ytest)

print("Accuracy:", test_accuracy) # should be larger than 90%
#print(yhat)

{0: 0.4166666666666667, 1: 0.4166666666666667, 2: 0.16666666666666666}
Accuracy: 0.9333333333333333


In [209]:
# All features of this dataset are binary
import pandas as pd
data = pd.read_csv('./datasets/binary_test.csv', header=None)
data = data.to_numpy()

X = data[:,1:]
y = data[:,0]
df = pd.DataFrame(data=X)
df.insert(16, 16, y)

#display(df)
N, D = X.shape
Ntrain = int(0.8 * N)
Xtrain = X[:Ntrain]
ytrain = y[:Ntrain]
Xtest = X[Ntrain:]
ytest = y[Ntrain:]


nbc = NBC(feature_types=['b'] * 16)
nbc.fit(Xtrain, ytrain)
yhat = nbc.predict(Xtest)
test_accuracy = np.mean(yhat == ytest)

print("Accuracy:", test_accuracy) # should be larger than 85%
#print(yhat)

{0: 0.5405405405405406, 1: 0.4594594594594595}
Accuracy: 0.8723404255319149


In [216]:
# All features of this dataset are categorical (bonus task)

data = pd.read_csv('./datasets/categorical_test.csv', header=None)
data = data.to_numpy()

#print(data.shape)
X = data[:,:-1]
y = data[:,-1]

N, D = X.shape
Ntrain = int(0.8 * N)
Xtrain = X[:Ntrain]
ytrain = y[:Ntrain]
Xtest = X[Ntrain:]
ytest = y[Ntrain:]


nbc = NBC(feature_types=['c'] * 9)
nbc.fit(Xtrain, ytrain)
yhat = nbc.predict(Xtest)
test_accuracy = np.mean(yhat == ytest)

print("Accuracy:", test_accuracy) # should be larger than 65%
#print(yhat) 

{0: 0.6929824561403509, 1: 0.30701754385964913}


AssertionError: 

## Logistic Regression

For logistic regression (LR), you should use the implementation in scikit-learn. Add the following
line to import the LR model.

In [32]:
from sklearn.linear_model import LogisticRegression

Check the scikit-learn documentation for the Logistic Regression model:
- http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
- http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression


## Comparing NBC and LR

### Experiments

The experiment is to compare the classification error of the NBC and LR trained on increasingly larger training datasets. 
Since the datasets are so small, you should do this multiple times and
average the classification error. One run should look as follows:
- Shuffle the data, put 20% aside for testing.
    
    ```N, D = X.shape
    Ntrain = int(0.8 * N)
    shuffler = np.random.permutation(N)
    Xtrain = X[shuffler[:Ntrain]]
    ytrain = y[shuffler[:Ntrain]]
    Xtest = X[shuffler[Ntrain:]]
    ytest = y[shuffler[Ntrain:]]
    
    ```  


- Train the classifiers with increasingly more data. For example, we can train classifiers with 10%, 20%, ..., 100% of the training data. For each case store the classification errors on the test set of the classifiers.

You may want to repeat this with at least 200 random permutations (possibly as large as 1000)
to average out the test error across the runs. In the end, you will get average test errors as a function of the size of the training data. 
We have written for you the function for making the plots for the experiments. 

In [33]:
# inputs:
#   nbc: Naive Bayes Classifier
#   lr: Logistic Regression Classifier
#   X, y: data
#   num_runs: we need repeat num_runs times and store average results
#   num_splits: we want to compare the two models on increasingly larger training sets.
#               num_splits defines the number of increasing steps. 
# outputs:
#   the arrays of the test errors across the runs of the two classifiers 
def compareNBCvsLR(nbc, lr, X, y, num_runs=200, num_splits=10):
    # The code below is just for compilation. 
    # You need to replace it by your own code.
    ###################################################
    ##### YOUR CODE STARTS HERE #######################
    ###################################################
    tst_errs_nbc = []
    tst_errs_lr = []
    
    return tst_errs_nbc, tst_errs_lr
    ###################################################
    ##### YOUR CODE ENDS HERE #########################
    ###################################################

In [34]:
def makePlot(tst_errs_nbc, tst_errs_lr, title=None, num_splits=10):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    ax.tick_params(axis='both', labelsize=20)

    ax.set_xlabel('Percent of training data used', fontsize=20)
    ax.set_ylabel('Classification Error', fontsize=20)
    if title is not None: ax.set_title(title, fontsize=25)

    xaxis_scale = [(i + 1) * (100/num_splits) for i in range(num_splits)]
    plt.plot(xaxis_scale, tst_errs_nbc, label='Naive Bayes')
    plt.plot(xaxis_scale, tst_errs_lr, label='Logistic Regression', linestyle='dashed')
    
    ax.legend(loc='upper right', fontsize=20)

### Datasets

Tasks: For each dataset,
1. prepare the data for the two classifiers, e.g., handle missing values, handle text/categorical data, etc.
2. compare the two classifiers on the dataset and generate the plots
3. write a short report of how you prepare the data and your observations of the comparison 

**Dataset 1: Iris Dataset**

https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html

In [35]:
# TODO: insert your code for experiments
###################################################
##### YOUR CODE STARTS HERE #######################
###################################################
from sklearn.datasets import load_iris
iris = load_iris()


###################################################
##### YOUR CODE ENDS HERE #########################
###################################################

**Dataset 2: Voting Dataset**

https://archive.ics.uci.edu/ml/datasets/congressional+voting+records

The logistic regression line meets the naive bayes line early in the plot. To see it clearer, you should use only 100 data points from the dataset for this experiment.

In [36]:
# load the dataset
# TODO: insert your code for experiments
###################################################
##### YOUR CODE STARTS HERE #######################
###################################################
voting = pd.read_csv('./datasets/voting.csv')

voting.info()

###################################################
##### YOUR CODE ENDS HERE #########################
###################################################

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   label                                   435 non-null    object
 1   handicapped-infants                     423 non-null    object
 2   water-project-cost-sharing              387 non-null    object
 3   adoption-of-the-budget-resolution       424 non-null    object
 4   physician-fee-freeze                    424 non-null    object
 5   el-salvador-aid                         420 non-null    object
 6   religious-groups-in-schools             424 non-null    object
 7   anti-satellite-test-ban                 421 non-null    object
 8   aid-to-nicaraguan-contras               420 non-null    object
 9   mx-missile                              413 non-null    object
 10  immigration                             428 non-null    object
 11  synfue